In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

column_names = [
    "Index",
    "Sample Size",
    "Learning Rate",
    "λ",
    "Train Erms",
    "Validate Erms",
    "Test Erms",
]
table_index = 0
table_df = pd.DataFrame(columns=column_names)
fig = ""
ax = ""

def build_and_train_model(train_x, train_y, val_x, val_y, learning_rate, lamda):
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # Define the model architecture
        # Build the model
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='tanh', kernel_regularizer=tf.keras.regularizers.L1(lamda), input_shape=(2,)),
            tf.keras.layers.Dense(1, activation='linear')
        ])
    
    # Compile the model with the specified learning rate
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Train the model
    result = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=100, batch_size=32, verbose=0)

    return model, result


def evaluate_model(model, x, y):

    loss = model.evaluate(x, y, verbose=0)

    y_pred = model.predict(x, verbose=0)

    return loss, y_pred

def add_data_to_table(sample_size, learning_rate, lamda, train_erms, validate_erms, test_erms):
    row_data = {}
    global table_df
    global table_index
    table_index = table_index + 1
    row_data["Index"] = table_index
    row_data["Sample Size"] = sample_size
    row_data["Learning Rate"] = learning_rate
    row_data["λ"] = lamda
    row_data["Train Erms"] = train_erms
    row_data["Validate Erms"] = validate_erms
    row_data["Test Erms"] = test_erms
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        table_df = pd.concat(
            [table_df, pd.DataFrame(row_data, index=[0])], ignore_index=True
        )


def print_table():
    print(table_df.to_string(index=False))

def get_N(x):
    N = len(x)
    return N

def get_erms(y, t):
    y_mse = tf.reduce_mean(tf.square(t-y))
    erms = np.sqrt(y_mse)
    return erms

def get_meshgrid(start_x1, stop_x1, start_x2, stop_x2, N):
    x1_range = np.linspace(start_x1, stop_x1, N)
    x2_range = np.linspace(start_x2, stop_x2, N)
    x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)
    
    input_grid = np.column_stack((x1_grid.ravel(), x2_grid.ravel()))

    return (x1_grid, x2_grid, input_grid)

def plot_loss(result, sample_size, learning_rate, lamda):

    # Plot training loss
    plt.plot(result.history['loss'], label='Training Loss')
    plt.plot(result.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plot_title = "MLFFNN Training and Validation Loss\nSampleSize = " + str(sample_size) + "\nLearningRate = "+str(learning_rate)+"\nLamda = "+str(lamda)
    plt.title(plot_title, fontsize=16, weight='bold')
    plt.legend()
    # plt.savefig("/home/dipendu/programs/mtech_2023/ml/ass2/trial/reg_2/"+plot_title+'.png')
    plt.show()

def scatter_plot_2d(x, y, x_label, y_label, plot_title, plot_color):
    plt.scatter(x, y, color=plot_color, label=plot_title, marker="*", s=50)
    plt.title(plot_title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    # plt.show()

def line_plot(x, y, plot_label, plot_color):
    plt.plot(x, y, color=plot_color, label=plot_label)
    plt.legend()
    # plt.show()

def plot_scatter(
    train_y, train_y_pred, val_y, val_y_pred, test_y, test_y_pred, sample_size, learning_rate, lamda
):
    scatter_plot_2d(train_y, train_y_pred, "Actual", "Predicted", 'Training Data', "blueviolet")

    scatter_plot_2d(val_y, val_y_pred, "Actual", "Predicted", 'Validation Data', "olivedrab")

    scatter_plot_2d(test_y, test_y_pred, "Actual", "Predicted", 'Test Data', "firebrick")

    min_point = min(
        np.min(train_y), np.min(train_y_pred), np.min(test_y), np.min(test_y_pred)
    )
    max_point = max(
        np.max(train_y), np.max(train_y_pred), np.max(test_y), np.max(test_y_pred)
    )

    line_plot((min_point, max_point), (min_point, max_point), "Guidance Line", "lightgray")

    plot_title = (
        "MLFFNN ScatterPlot\nSampleSize = " + str(sample_size) + "\nLearningRate = " + str(learning_rate) + "\nLamda = "+str(lamda)
    )
    # plt.savefig("/home/dipendu/programs/mtech_2023/ml/ass2/trial/reg_2/"+plot_title+'.png')
    plt.title(plot_title, fontsize=16, weight='bold')
    plt.show()

def plot_data(x, y, pred_y, x1_grid, x2_grid, y_pred_surface, x_color, y_color, scatter_label, plot_label, x_label, y_label, z_label, plot_title, legend):
    
    global fig
    fig = plt.figure()
    global ax
    ax = fig.add_subplot(111, projection="3d")
    scatter = ax.scatter(x[:, 0], x[:, 1], y, color=x_color, label=scatter_label)
    ax.plot_surface(x1_grid, x2_grid, y_pred_surface, color=y_color, alpha=0.5, label=plot_label)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_zlabel(z_label)
    plot_title = plot_title + legend
    ax.set_title(plot_title, fontsize=16, weight='bold')
    ax.legend([scatter], [legend])
    # plt.savefig("/home/dipendu/programs/mtech_2023/ml/ass2/trial/reg_2/"+plot_title+'.png')
    plt.show()

def plot_all_data(model, train_x, train_y, train_y_pred, val_x, val_y, val_y_pred, test_x, test_y, test_y_pred, sample_size, learning_rate, lamda):
    
    mesh_count = 100

    # Predict outputs for training data
    min_x1 = min(np.min(train_x[0]), np.min(val_x[0]), np.min(test_x[0]))-2
    max_x1 = max(np.max(train_x[0]), np.max(val_x[0]), np.max(test_x[0]))+2
    min_x2 = min(np.min(train_x[1]), np.min(val_x[1]), np.min(test_x[1]))-2
    max_x2 = max(np.max(train_x[1]), np.max(val_x[1]), np.max(test_x[1]))+2
    x1_grid, x2_grid, input_grid = get_meshgrid(min_x1, max_x1, min_x2, max_x2, mesh_count)
    
    # Plotting
    # fig = plt.figure(figsize=(15, 5))

    title = "MLFFNN Approximated Function\nSampleSize = " + str(sample_size) + "\nLearningRate = " + str(learning_rate) + "\nLamda = "+str(lamda)+"\nOn "

    # Plot the approximated functions obtained using training data
    # ax1 = fig.add_subplot(131, projection='3d')
    y_pred_train_surface = model.predict(input_grid, verbose=0).reshape(x1_grid.shape)
    # plot_data(train_x, train_y, train_y_pred, x1_grid, x2_grid, y_pred_train_surface, 'blue', 'orange', 'Training Data', 'Approximated Function', 'X1', 'X2', 'Y', title+'Training Data', ax1)
    plot_data(train_x, train_y, train_y_pred, x1_grid, x2_grid, y_pred_train_surface, 'blue', 'orange', 'Training Data', 'Approximated Function', 'X1', 'X2', 'Y', title, 'Training Data')

    # Plot the approximated functions obtained using validation data
    # ax2 = fig.add_subplot(132, projection='3d')
    y_pred_val_surface = model.predict(input_grid, verbose=0).reshape(x1_grid.shape)
    # plot_data(val_x, val_y, val_y_pred, x1_grid, x2_grid, y_pred_val_surface, 'red', 'green', 'Validation Data', 'Approximated Function', 'X1', 'X2', 'Y', title+'Validation Data', ax2)
    plot_data(val_x, val_y, val_y_pred, x1_grid, x2_grid, y_pred_val_surface, 'red', 'green', 'Validation Data', 'Approximated Function', 'X1', 'X2', 'Y', title, 'Validation Data')
    
    # Plot the approximated functions obtained using test data
    # ax3 = fig.add_subplot(133, projection='3d')
    y_pred_test_surface = model.predict(input_grid, verbose=0).reshape(x1_grid.shape)
    # plot_data(test_x, test_y, test_y_pred, x1_grid, x2_grid, y_pred_test_surface, 'purple', 'yellow', 'Test Data', 'Approximated Function', 'X1', 'X2', 'Y', title + 'Test Data', ax3)
    plot_data(test_x, test_y, test_y_pred, x1_grid, x2_grid, y_pred_test_surface, 'purple', 'yellow', 'Test Data', 'Approximated Function', 'X1', 'X2', 'Y', title, 'Test Data')

    # plt.tight_layout()
    # plt.show()


def plot_graphs(train_x, train_y, val_x, val_y, test_x, test_y, sample_size,  regularization_coefficients, learning_rate=0.1):

    for lamda in regularization_coefficients:
        model, result = build_and_train_model(train_x, train_y, val_x, val_y, learning_rate, lamda)
        
        train_loss, train_y_pred = evaluate_model(model, train_x, train_y)
        val_loss, val_y_pred = evaluate_model(model, val_x, val_y)
        test_loss, test_y_pred = evaluate_model(model, test_x, test_y)

        train_erms = get_erms(train_y_pred, train_y)
        validate_erms = get_erms(val_y_pred, val_y)
        test_erms = get_erms(test_y_pred, test_y)
        
        # Plot all the graphs
        plot_all_data(model, train_x, train_y, train_y_pred, val_x, val_y, val_y_pred, test_x, test_y, test_y_pred, sample_size, learning_rate, lamda)

        # Plot scatter the graphs
        plot_scatter(train_y, train_y_pred, val_y, val_y_pred, test_y, test_y_pred, sample_size, learning_rate, lamda)
        
        # Plot training loss
        plot_loss(result, sample_size, learning_rate, lamda)

        add_data_to_table(sample_size, learning_rate, lamda, train_erms, validate_erms, test_erms)

        # Print train loss
        print("Train Loss:", train_loss)
        # Print validation loss
        print("Validation Loss:", val_loss)
        # Print test loss
        print("Test Loss:", test_loss)
    

def main():
    folder_number = "9"
    current_directory = os.getcwd()
    # regression_dataset_2_path=current_directory+ "/Datasets_for_A1/Regression/Dataset 2/" + folder_number + "/"
    regression_dataset_2_path = (
        "/home/dipendu/programs/mtech_2023/ml/ass2/Datasets_for_A1/Regression/Dataset 2/"
        + folder_number
        + "/"
    )

    regression_dataset_2_Train_Sample_1 = (
        regression_dataset_2_path + "train50_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_Train_Sample_1)
    data = df.to_numpy()
    sorted_data = data[data[:, 1].argsort()]
    train_x_1 = sorted_data[:, :2]
    train_y_1 = sorted_data[:, 2]
    train_x_1 = train_x_1

    regression_dataset_2_Train_Sample_2 = (
        regression_dataset_2_path + "train200_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_Train_Sample_2)
    data = df.to_numpy()
    sorted_data = data[data[:, 1].argsort()]
    train_x_2 = sorted_data[:, :2]
    train_y_2 = sorted_data[:, 2]
    train_x_2 = train_x_2

    regression_dataset_2_validation = (
        regression_dataset_2_path + "val_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_validation)
    data = df.to_numpy()
    sorted_data = data[data[:, 1].argsort()]
    val_x = sorted_data[:, :2]
    val_y = sorted_data[:, 2]
    val_x = val_x

    regression_dataset_2_test = (
        regression_dataset_2_path + "test_" + folder_number + ".csv"
    )
    df = pd.read_csv(regression_dataset_2_test)
    data = df.to_numpy()
    sorted_data = data[data[:, 1].argsort()]
    test_x = data[:, :2]
    test_y = data[:, 2]
    test_x = test_x

    learning_rates = [0.01]
    regularization_coefficients = [0.0, 0.0001, 1e-6, 1e-9]

    sample_size = get_N(train_x_1)
    for learning_rate in learning_rates:
        plot_graphs(
            train_x_1,
            train_y_1,
            val_x,
            val_y,
            test_x,
            test_y,
            sample_size,
            regularization_coefficients,
            learning_rate,
        )

    sample_size = get_N(train_x_2)
    for learning_rate in learning_rates:
        plot_graphs(
            train_x_2,
            train_y_2,
            val_x,
            val_y,
            test_x,
            test_y,
            sample_size,
            regularization_coefficients,
            learning_rate,
        )

    print_table()


if __name__ == "__main__":
    main()